Implement fuzzy matching on LLCs that have filed evictions to lookup registered LLC info

Updated 4/4/25

In [ ]:
import pandas as pd
import numpy as np
import re
from concurrent.futures import ProcessPoolExecutor
import unicodedata
from rapidfuzz import process, fuzz

In [ ]:
evictions_df = pd.read_parquet('DATA/evictions.parquet')

In [ ]:
llc_df = pd.read_parquet('DATA/llc.parquet')

Conservative pre-processing approach to maintain efficiency

Can make this more aggressive if it doesn't work out

In [19]:
def preprocess_names(df, column_name):
    """
    Normalize names, sort words alphabetically for natural language processing if we need it later
    @df: pandas dataframe
    @column_name: str column name to preprocess
    
    """
    column = column_name + "_normalized"
    df[column] = (
        df[column_name]
        .str.lower()
        .str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)
    ) 

    df[column] = (
        df[column]
        .str.split()
        .map(lambda x: " ".join(sorted(x)) if isinstance(x, list) else ' ')
    )

    return df

In [ ]:
e_columns_for_p = ['plaintiff_name'] # columns for preprocessing, add more as needed
l_columns_for_p = ['EntityID']

In [ ]:
for i in e_columns_for_p: # apply preprocessing for both df
    evictions_df = preprocess_names(evictions_df, i)

for i in l_columns_for_p:
    llc_df = preprocess_names(llc_df, i)

In [ ]:
# create flag 'is_llc' to restrict lookup to only llcs
llc_keywords = r"\b(llc|l\.l\.c|inc|inc\.|corporation|corp|corp\.|co|co\.|company|ltd|ltd\.|lp|l\.p\.|pllc|plc|plc\.|limited|limited liability company)\b"
evictions_df["is_llc"] = evictions_df["plaintiff_name_normalized"].str.contains(llc_keywords, regex=True)

/tmp/ipykernel_759534/2018107948.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  evictions["is_llc"] = evictions["plaintiff_name_normalized"].str.contains(llc_keywords, regex=True)


In [6]:
evictions_df = evictions_df[evictions_df['is_llc']] # restrict lookup to only llcs
plaintiffs = set(evictions_df['plaintiff_name_normalized']) # set of unique LLC plaintiffs
llcs = set(llc_df['EntityID_normalized']) # set of unique registered LLCs

In [7]:
# Function to process matching for a subset of plaintiff names
def process_chunk(plaintiff_chunk, llc_names, confidence=80):
    matches = {}
    for i, plaintiff in enumerate(plaintiff_chunk):
        best_match, score = process.extractOne(plaintiff, llc_names, scorer=fuzz.WRatio)
        if best_match >= confidence:
            matches[plaintiff] = best_match[0]
        else:
            matches[plaintiff] = None
        if (i + 1) % 100 == 0:
            print(f"Processed {i + 1} out of {len(plaintiff_chunk)} plaintiffs in this chunk.")
    return matches

# Function to split the plaintiff set and process in parallel
def parallel_match_plaintiffs(plaintiff_names_set, llc_names_set, chunk_size=1000):
    plaintiff_chunks = [list(plaintiff_names_set)[i:i + chunk_size] for i in range(0, len(plaintiff_names_set), chunk_size)]
    print(f"Total of {len(plaintiff_chunks)} chunks to process.")

    all_matches = {}
    with ProcessPoolExecutor() as executor:
        futures = [executor.submit(process_chunk, chunk, llc_names_set) for chunk in plaintiff_chunks]
        for i, future in enumerate(futures):
            all_matches.update(future.result())
            print(f"Chunk {i + 1} processed. Total matches found: {len(all_matches)}")
    
    return all_matches

In [8]:
chunk_size = len(plaintiffs) // 16 # adjust for num of cores, 16 worked well under an hour
# probably only needed ~100GB memory or less

In [9]:
matches = parallel_match_plaintiffs(plaintiffs, llcs, chunk_size=chunk_size)

Total of 17 chunks to process.
Processed 100 out of 973 plaintiffs in this chunk.
Processed 100 out of 973 plaintiffs in this chunk.
Processed 100 out of 973 plaintiffs in this chunk.
Processed 100 out of 973 plaintiffs in this chunk.
Processed 100 out of 973 plaintiffs in this chunk.
Processed 100 out of 973 plaintiffs in this chunk.
Processed 100 out of 973 plaintiffs in this chunk.
Processed 100 out of 973 plaintiffs in this chunk.
Processed 100 out of 973 plaintiffs in this chunk.
Processed 100 out of 973 plaintiffs in this chunk.
Processed 100 out of 973 plaintiffs in this chunk.
Processed 100 out of 973 plaintiffs in this chunk.
Processed 100 out of 973 plaintiffs in this chunk.
Processed 100 out of 973 plaintiffs in this chunk.
Processed 100 out of 973 plaintiffs in this chunk.
Processed 100 out of 973 plaintiffs in this chunk.
Processed 200 out of 973 plaintiffs in this chunk.
Processed 200 out of 973 plaintiffs in this chunk.
Processed 200 out of 973 plaintiffs in this chunk.


In [10]:
for plaintiff, llc in list(matches.items())[:10]:
    print(f"{plaintiff} -> {llc}")

and cb inc rentals sales -> and building llc remodeling springbuck
ann commons lp wingfield -> ann commons llc wingfield
lp stuart woods -> crystal llc owner woods
adirondack llc -> adirondack llc
llc mhci wap -> llc m
apts creeks edge llc -> edge llc
cedar gri llc park -> cedar gri llc park
estates llc mobile riverview -> estates llc mobile riverview
llc two wh -> llc two wh
college lp mission nrha -> college llc mission nrha


In [11]:
evictions_df['best_match'] = evictions_df['plaintiff_name_normalized'].map(matches)

In [22]:
evictions_df = pd.merge(
    left=evictions_df,
    right=llc_df,
    left_on='best_match',
    right_on='EntityID_normalized',
    how='left'
).drop(columns=['best_match'])

In [32]:
evictions.to_parquet('evictions_matched.parquet')